# アソシエーション分析

### Apriori

## データ読み込み

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd

In [2]:
df = pd.read_csv('../data/Apriori/groceries.csv')
df

,cardid,goods
0,39808,freshmeat
1,39808,dairy
2,39808,confectionery
3,67362,freshmeat
4,67362,confectionery
5,10872,cannedveg
6,10872,frozenmeal
7,10872,beer
8,10872,fish
9,28935,fruitveg


userとitemの数

In [3]:
# usersの種類
users = sorted(df['cardid'].unique())
# itemの種類
items = sorted(df['goods'].unique())

print(f'usersの種類数：{len(users)}\nitemsの種類数：{len(items)}')

usersの種類数：5
itemsの種類数：8


## 前処理

### データを「（ユーザが買った雑貨の）リストのリスト」という形式に変換する

In [4]:
def items_to_list(data, user_id):
    """あるユーザが買った雑貨を一つのリストに変換します.
    
    Args:
        data (DataFrame): ユーザーが雑貨を買ったデータ
        user_id (str)： ユーザーID

    Returns:
        (list): あるユーザ（ユーザーID）が買った雑貨のリスト.

    """
    item_list = data.loc[data['cardid']==user_id]['goods'].tolist()
    return item_list

In [5]:
# 全ユーザに対して各自の雑貨リストを計算して、「（ユーザが買った雑貨の）リストのリスト」を作成する

from functools import partial
# 訓練データ
func = partial(items_to_list, df)
item_list_list = list(map(func, users))
item_list_list

[['cannedveg', 'frozenmeal', 'beer', 'fish'],
 ['fruitveg', 'frozenmeal'],
 ['freshmeat', 'dairy', 'confectionery'],
 ['fruitveg', 'fish'],
 ['freshmeat', 'confectionery']]

## モデリング

### mlxtend パッケージを利用する

In [6]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()

# 「（ユーザが買った雑貨の）リストのリスト」を「sparse format」に変換する
te_ary = te.fit(item_list_list).transform(item_list_list)
item_sparse = pd.DataFrame(te_ary, columns=te.columns_)

In [7]:
# 頻出アイテム集合の検出
# supportが0.2以下のアイテムを削除する場合
frequent_itemsets = apriori(item_sparse, min_support=0.2, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.2,(beer)
1,0.2,(cannedveg)
2,0.4,(confectionery)
3,0.2,(dairy)
4,0.4,(fish)


In [8]:
#相関ルールの検出

from mlxtend.frequent_patterns import association_rules
# confidenceが0.4以下のルールを削除する場合
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)

In [9]:
# 検出されたルールをさらに絞る
# 条件１：lift値が2以上
# 条件2：support値が0.2以上
rules_final = rules[(rules['lift']>=2) & (rules['support']>=0.2)]
rules_final

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(beer),(cannedveg),0.2,0.2,0.2,1.0,5.0,0.16,inf
1,(cannedveg),(beer),0.2,0.2,0.2,1.0,5.0,0.16,inf
2,(beer),(fish),0.2,0.4,0.2,1.0,2.5,0.12,inf
3,(fish),(beer),0.4,0.2,0.2,0.5,2.5,0.12,1.600000
4,(beer),(frozenmeal),0.2,0.4,0.2,1.0,2.5,0.12,inf
5,(frozenmeal),(beer),0.4,0.2,0.2,0.5,2.5,0.12,1.600000
6,(cannedveg),(fish),0.2,0.4,0.2,1.0,2.5,0.12,inf
7,(fish),(cannedveg),0.4,0.2,0.2,0.5,2.5,0.12,1.600000
8,(cannedveg),(frozenmeal),0.2,0.4,0.2,1.0,2.5,0.12,inf
9,(frozenmeal),(cannedveg),0.4,0.2,0.2,0.5,2.5,0.12,1.600000


In [10]:
# ルールを出力する
def print_rule(rule_index):
    """特定行のルールを出力する.
    
    Args:
        rule_index (int): 何行目のルールを出力するか.

    Returns:
        (None)

    """
    # antecedentsを取得
    items1 = rules_final.iloc[rule_index]['antecedents']
    st1 = ''
    for i in items1:
        st1 += f'「{i}」、'
    st1 = st1[:-1]
    # consequentsを取得
    items2 = list(rules_final.iloc[rule_index]['consequents'])
    st2 = ''
    for i in items2:
        st2 += f'「{i}」、'
    st2 = st2[:-1]

    print(f'相関ルール１： \n\t{st1} を買った人が　{st2} を買う')
    print(f'指標結果：\n\tSupport: {rules_final.iloc[rule_index]["support"]:.3}\n\tConfidence: {rules_final.iloc[rule_index]["confidence"]:.3}\n\tLift　: {rules_final.iloc[rule_index]["lift"]:.3}')
    print(f'\tLeverage: {rules_final.iloc[rule_index]["leverage"]:.3}\n\tConviction: {rules_final.iloc[rule_index]["conviction"]:.3}')

In [11]:
# １行目のルールを確認する
print_rule(rule_index=0)

相関ルール１： 
	「beer」 を買った人が　「cannedveg」 を買う
指標結果：
	Support: 0.2
	Confidence: 1.0
	Lift　: 5.0
	Leverage: 0.16
	Conviction: inf


In [12]:
# 3行目のルールを確認する
print_rule(rule_index=2)

相関ルール１： 
	「beer」 を買った人が　「fish」 を買う
指標結果：
	Support: 0.2
	Confidence: 1.0
	Lift　: 2.5
	Leverage: 0.12
	Conviction: inf


### 推薦項目のリストを作成する

In [13]:
from functools import reduce 
def predict(users, item_list_list):
    """推薦項目のリストを作成する.
    
    Args:
        user_id (str): ユーザーID.
        item_list (list): あるユーザーが買った雑貨リスト.

    Returns:
        (DataFrame): このユーザーに推薦する雑貨.

    """    
    global rules_final
    predict_result = pd.DataFrame(None)
    
    for i in range(len(users)):
        user_id = users[i]
        item_list = set(item_list_list[i])

        predict = rules_final[rules_final['antecedents'] <= item_list] ['consequents'].unique()
        # 推薦項目がない 
        if len(predict) == 0:
            continue
            
        predict = set(reduce(lambda x, y: x | y, predict))    
        
        # 推薦項目がない 
        if len(predict) == 0:
            continue
        
        # すでに買った雑貨を削除して推薦しない
        predict_item = predict&item_list
        if len(predict_item) >0:
            for item in predict_item:
                predict.remove(item)
        
        # 推薦項目がない 
        if len(predict) == 0:
            continue
 
        # 推薦項目がある
        predict = pd.DataFrame(list(predict), columns=['item_id'])
        predict = predict.assign(user_id = user_id)
        predict_result = pd.concat([predict_result, predict], axis=0)

    predict_result.reset_index(drop=True, inplace=True)
    return predict_result

# レコメンドリスト作成
prediction = predict(users, item_list_list)

In [14]:
user_id = 67362
print(f'\nuser_id: {user_id} に対するレコメンドリスト')
prediction[prediction['user_id'] == user_id]


user_id: 67362 に対するレコメンドリスト


,item_id,user_id
6,dairy,67362
